# Notebook for CTM Experimentation

## Load libraries

In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append("../")
from src import puhti_files
from src.common_basis import *

Database password (connection attempt failed with (pymysql.err.OperationalError) (1045, "Access denied for user 'hpc-hd-ro'@'puhti-nat-1.csc.fi' (using password: NO)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)): ········


In [2]:
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install gensim

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

In [4]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /users/sebarodr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /users/sebarodr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
!{sys.executable} -m pip install contextualized-topic-models
!{sys.executable} -m pip install pyldavis

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [6]:
from gensim.models import Word2Vec

In [7]:
SEED = 42
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

In [8]:
!{sys.executable} -m pip install --upgrade contextualized-topic-models

Defaulting to user installation because normal site-packages is not writeable


## Dev Data

In [9]:
df = puhti_files.genre_data_to_pandas(data="dev")

Read in dataset dev.csv. Set param 'data' to 'dev', 'test' or 'train' if you want another dataset.


In [10]:
df['text'] = df['document_id'].apply(lambda x: puhti_files.read_text_file(x))

In [11]:
df['text']

0       BIOGRAPHICAL SKETCHES\n\nOF SOME OF THF\nLEADI...
1       i/\n\n//\n\n-I,\nI,\n\n/· / t\n\nJiY\n\n1;-f\n...
2       A SO·3 RM~l\n\nOF~\n\nPRAY'nER and TH~gANPKSG ...
3       ··Y· ·····\n\n· _ ·1\n\n__·\n\nri_·_ i\n\n"· ·...
4       ,-* U* A.U S..,\nZ.T -ES-AU'/iS\n\nLingux Lati...
                              ...                        
3007    brol\n\ntr\n\n71Z~~''\n: 'E- 1 1\n\n~~~~~~~~ '...
3008    THE\nW AY\n\nTO\nK E E P HIM.\n\nA\nC -O'M E D...
3009    VOLUME V.\n\nOF THE\nAUTHOR'S WORKS.\n\nCONTAI...
3010    A D V I C E\nADVICE\n\nTO THE\nPEOPLE in GENER...
3011    / /\nT H F-\nTHE\nMAID OF BATH,\n\nA\nCO M E D...
Name: text, Length: 3012, dtype: object

In [12]:
df['text'] = df['text'].str.lower()

In [13]:
df['text'] = df["text"].str.replace('\s+', ' ', regex=True)

In [14]:
df['text'] = df['text'].str.replace(r'[^A-Za-z0-9]+', ' ', regex=True)

In [15]:
df['text'] = df["text"].str.replace('\s+', ' ', regex=True)

In [16]:
df['text'] = df['text'].str.replace('\d+', '',regex=True)

## Metadata

In [17]:
metadata = puhti_files.ecco_metadata_to_pandas()

In [18]:
metadata

,document_id_octavo,publication_year,gatherings,total_price,publication_place,author_id,other_actors
0,0000100100,1723,12mo,NaN,London,98354271,bbti_77907; bbti_72802; 51557323
1,0000100200,1733,8vo,NaN,London,76461649,66340504; bbti_51500; 68786630
2,0000100300,1765,12mo,NaN,London,46318688,bbti_112410; bbti_26196
3,0000100400,1720,12mo,NaN,London,67409380,bbti_4638; bbti_70361
4,0000100501,1703,4to,NaN,London,61784573,20781965; bbti_70360
...,...,...,...,...,...,...,...
275222,1143993,1680,2fo,NaN,London,NaN,bbti_47378
275223,1143994,1678,8vo,NaN,London,66813373,bbti_44765; bbti_42013
275224,1143997,1685,2fo,NaN,Dublin,NaN,samuelhelsham_0; 147809350; andrewcrook_1; 235...
275225,1144469,1689,2fo,NaN,London,NaN,NaN


In [19]:
dict_labels = puhti_files.labels_to_dict()

In [20]:
ecco_core = pd.read_sql_table("ecco_core", con)

In [21]:
ecco_core.head()

,ecco_id,estc_id,ecco_part,ecco_module,ecco_full_title,estc_id_octavo,ecco_nr_characters,ecco_nr_tokens,ecco_date_start,ecco_date_end,ecco_pages,ecco_languge,ecco_nr_paragraphs
0,0041606600,T1676,ecco1,Social Sciences,"The abuse of standing parliaments, and the gre...",T001676,52406.0,8876.0,17500101.0,17509999.0,30.0,English,114.0
1,0296902200,T54193,ecco1,Religion and Philosophy,"Reformation, or Mockery, Argued, from the gene...",T054193,49270.0,8876.0,17290101.0,17299999.0,32.0,English,107.0
2,1072500700,N8457,ecco1,Social Sciences,A defence of the rights of the Dock Company at...,N008457,53298.0,8876.0,17870101.0,17879999.0,45.0,English,150.0
3,1180701700,T46647,ecco1,Social Sciences,A refutation of the letter to an Honble. Briga...,T046647,50469.0,8876.0,17600101.0,17609999.0,55.0,English,230.0
4,1297204300,T214687,ecco1,Law,Answers for Alexander Alison deputy receiver g...,T214687,50188.0,8876.0,17650101.0,17659999.0,28.0,English,175.0


In [22]:
pd.merge(df, metadata, left_on='document_id', right_on='document_id_octavo')

,document_id,work_id,main_category,sub_category,text,document_id_octavo,publication_year,gatherings,total_price,publication_place,author_id,other_actors
0,1243400400,X-biographical sketches of some of the leading...,4,10,biographical sketches of some of thf leadin me...,1243400400,1793,8vo,NaN,Edinburgh,NaN,jamesdickson_0; 49278054; jjfairbairn_0
1,1588400300,19754-rudiments and practical exercises for le...,1,3,i i i t jiy f s ra a iid oi c i yic f r...,1588400300,1798,8vo,NaN,Edinburgh,78653705,grobinsonandcompany_0; 49278054
2,1628902300,1-liturgies,9,29,a so rm l of pray ner and th ganpksg iv ingg ...,1628902300,1797,4to,NaN,London,135203514,53835804; 100093484
3,1404100101,1519-new history of holy bible,4,9,y ri i u f h r r z p i i r tr j i j i ci l...,1404100101,1796,8vo,NaN,Glasgow,98362473,4730486
4,0964200401,1653-thesaurus linguae latinae compendiarius,1,3,u a u s z t es au is lingux latin e compendia...,0964200401,1736,4to,NaN,London,76653963,36768263; bbti_108117; 121291376; 12361047; bb...
...,...,...,...,...,...,...,...,...,...,...,...,...
3080,1046100100,3104-gesundheits-katechismus,3,6,brol tr z e i dl h x catechism o f health se...,1046100100,1797,12mo,NaN,Edinburgh,9891855,angusandson_0; jburnet_0; dunlopandwilsonbrash...
3081,1207401900,1022-way to keep him,0,1,the w ay to k e e p him a c o m e d y in thre ...,1207401900,1760,12mo,NaN,London,24641385,76551088
3082,0098000205,61-works,6,25,volume v of the author s works containing the ...,0098000205,1741,8vo,NaN,Dublin,14777110,34511788
3083,0884900100,983-avis au peuple,3,6,a d v i c e advice to the people in general w ...,0884900100,1768,8vo,NaN,London,98841872,bbti_5155; bbti_19333


In [23]:
ecco_wdata = pd.merge(df, ecco_core, left_on='document_id', right_on='ecco_id')

In [24]:
ecco_wdata['ecco_languge'].value_counts()

English               2851
Latin                   73
French                  73
Italian                  6
Welsh                    5
Multiple languages       2
Spanish                  1
Manx                     1
Name: ecco_languge, dtype: int64

In [25]:
ecco_wdata[ecco_wdata['ecco_languge'] == 'English'].head()

,document_id,work_id,main_category,sub_category,text,ecco_id,estc_id,ecco_part,ecco_module,ecco_full_title,estc_id_octavo,ecco_nr_characters,ecco_nr_tokens,ecco_date_start,ecco_date_end,ecco_pages,ecco_languge,ecco_nr_paragraphs
0,1243400400,X-biographical sketches of some of the leading...,4,10,biographical sketches of some of thf leadin me...,1243400400,N31961,ecco1,Social Sciences,Biographical sketches of some of the leading m...,N031961,75924.0,13037.0,17930101.0,17939999.0,62.0,English,226.0
1,1588400300,19754-rudiments and practical exercises for le...,1,3,i i i t jiy f s ra a iid oi c i yic f r...,1588400300,N13159,ecco2,Literature and Language,"Rudiments and practical exercises, for learnin...",N13159,521842.0,93010.0,17980000.0,17989999.0,367.0,English,10396.0
2,1628902300,1-liturgies,9,29,a so rm l of pray ner and th ganpksg iv ingg ...,1628902300,T190203,ecco2,Religion and Philosophy,A form of prayer and thanksgiving to almighty ...,T190203,12039.0,2086.0,17970000.0,17979999.0,11.0,English,126.0
3,1404100101,1519-new history of holy bible,4,9,y ri i u f h r r z p i i r tr j i j i ci l...,1404100101,N20172,ecco2,Religion and Philosophy,"A new history of the Holy Bible, from the begi...",N20172,1254613.0,216241.0,17960000.0,17969999.0,489.0,English,5065.0
4,0964200401,1653-thesaurus linguae latinae compendiarius,1,3,u a u s z t es au is lingux latin e compendia...,0964200401,T65492,ecco1,Literature and Language,"Thesaurus linguæ Latinæ compendiarius: or, a c...",T065492,6144591.0,1024035.0,17360101.0,17369999.0,674.0,English,14843.0


In [26]:
ecco_document_parts = pd.read_sql_table("ecco_document_parts", con)

In [28]:
ecco_document_parts.head()

,ecco_id,document_part_type,document_part_number,document_part_id,document_part_nr_characters,document_part_nr_tokens
0,0052700800,frontmatter,1.0,487392.0,560.0,102.0
1,0052700800,titlePage,2.0,487393.0,712.0,146.0
2,0052700800,frontmatter,3.0,487394.0,154.0,24.0
3,0052700800,body,4.0,487395.0,143691.0,25360.0
4,0052700800,backmatter,5.0,487404.0,509.0,99.0


In [29]:
ecco_wdata.head()

,document_id,work_id,main_category,sub_category,text,ecco_id,estc_id,ecco_part,ecco_module,ecco_full_title,estc_id_octavo,ecco_nr_characters,ecco_nr_tokens,ecco_date_start,ecco_date_end,ecco_pages,ecco_languge,ecco_nr_paragraphs
0,1243400400,X-biographical sketches of some of the leading...,4,10,biographical sketches of some of thf leadin me...,1243400400,N31961,ecco1,Social Sciences,Biographical sketches of some of the leading m...,N031961,75924.0,13037.0,17930101.0,17939999.0,62.0,English,226.0
1,1588400300,19754-rudiments and practical exercises for le...,1,3,i i i t jiy f s ra a iid oi c i yic f r...,1588400300,N13159,ecco2,Literature and Language,"Rudiments and practical exercises, for learnin...",N13159,521842.0,93010.0,17980000.0,17989999.0,367.0,English,10396.0
2,1628902300,1-liturgies,9,29,a so rm l of pray ner and th ganpksg iv ingg ...,1628902300,T190203,ecco2,Religion and Philosophy,A form of prayer and thanksgiving to almighty ...,T190203,12039.0,2086.0,17970000.0,17979999.0,11.0,English,126.0
3,1404100101,1519-new history of holy bible,4,9,y ri i u f h r r z p i i r tr j i j i ci l...,1404100101,N20172,ecco2,Religion and Philosophy,"A new history of the Holy Bible, from the begi...",N20172,1254613.0,216241.0,17960000.0,17969999.0,489.0,English,5065.0
4,0964200401,1653-thesaurus linguae latinae compendiarius,1,3,u a u s z t es au is lingux latin e compendia...,0964200401,T65492,ecco1,Literature and Language,"Thesaurus linguæ Latinæ compendiarius: or, a c...",T065492,6144591.0,1024035.0,17360101.0,17369999.0,674.0,English,14843.0


In [30]:
ecco_parts_data = pd.merge(ecco_wdata, ecco_document_parts, left_on='document_id', right_on='ecco_id')

In [31]:
ecco_parts_data['document_part_type'].value_counts()

frontmatter    4482
body           4224
titlePage      4154
backmatter     1294
TOC            1180
index           576
Name: document_part_type, dtype: int64

In [32]:
ecco_parts_data[ecco_parts_data['document_part_type']=='body'].head()

,document_id,work_id,main_category,sub_category,text,ecco_id_x,estc_id,ecco_part,ecco_module,ecco_full_title,...,ecco_date_end,ecco_pages,ecco_languge,ecco_nr_paragraphs,ecco_id_y,document_part_type,document_part_number,document_part_id,document_part_nr_characters,document_part_nr_tokens
2,1243400400,X-biographical sketches of some of the leading...,4,10,biographical sketches of some of thf leadin me...,1243400400,N31961,ecco1,Social Sciences,Biographical sketches of some of the leading m...,...,17939999.0,62.0,English,226.0,1243400400,body,3.0,214495.0,73568.0,12607.0
7,1588400300,19754-rudiments and practical exercises for le...,1,3,i i i t jiy f s ra a iid oi c i yic f r...,1588400300,N13159,ecco2,Literature and Language,"Rudiments and practical exercises, for learnin...",...,17989999.0,367.0,English,10396.0,1588400300,body,4.0,751685.0,517000.0,92167.0
10,1628902300,1-liturgies,9,29,a so rm l of pray ner and th ganpksg iv ingg ...,1628902300,T190203,ecco2,Religion and Philosophy,A form of prayer and thanksgiving to almighty ...,...,17979999.0,11.0,English,126.0,1628902300,body,3.0,675368.0,11346.0,1978.0
14,1404100101,1519-new history of holy bible,4,9,y ri i u f h r r z p i i r tr j i j i ci l...,1404100101,N20172,ecco2,Religion and Philosophy,"A new history of the Holy Bible, from the begi...",...,17969999.0,489.0,English,5065.0,1404100101,body,4.0,652036.0,973521.0,168107.0
17,0964200401,1653-thesaurus linguae latinae compendiarius,1,3,u a u s z t es au is lingux latin e compendia...,0964200401,T65492,ecco1,Literature and Language,"Thesaurus linguæ Latinæ compendiarius: or, a c...",...,17369999.0,674.0,English,14843.0,0964200401,body,3.0,443678.0,4124072.0,669054.0


## Preprocessing 

This preprocessing will be done only over the text in english, so we can apply the stopwords and don't confuse the model with multiple languages.

In [33]:
english_docs = ecco_parts_data[(ecco_parts_data['ecco_languge'] == 'English') & (ecco_parts_data['document_part_type'] == 'body')]

In [35]:
english_docs.shape

(4052, 24)

In [36]:
def clean_text_tokens(text,tokenizer,stopwords):
    tokens = tokenizer(text)
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    tokens = [t for t in tokens if not t in stopwords]
    #tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    return tokens

In [38]:
def clean_text_gtext(text,tokenizer,stopwords):
    tokens = tokenizer(text)
    tokens = [t for t in tokens if len(t) > 2]  # Remove short tokens
    tokens = [t for t in tokens if not t in stopwords]
    
    return " ".join(tokens)

In [39]:
stop_words = stopwords.words("english")
english_docs["clean_t"] =english_docs["text"].map(lambda x: clean_text_gtext(x, word_tokenize, stop_words))

<ipython-input-39-1461aaa3bfae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_docs["clean_t"] =english_docs["text"].map(lambda x: clean_text_gtext(x, word_tokenize, stop_words))


In [40]:
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords

In [41]:
english_docs['text']

2        biographical sketches of some of thf leadin me...
7        i i i t jiy  f     s ra a iid oi c i  yic  f r...
10       a so  rm l of pray ner and th ganpksg iv ingg ...
14        y  ri i u  f h r r z  p i i r tr j i j i ci l...
17        u a u s z t es au is lingux latin e compendia...
                               ...                        
15890    brol tr z e   i dl h x catechism o f health se...
15893    the w ay to k e e p him a c o m e d y in thre ...
15898    volume v of the author s works containing the ...
15904    a d v i c e advice to the people in general w ...
15908     t h f the maid of bath a co m e d y f t hr ee...
Name: text, Length: 4052, dtype: object

In [42]:
english_docs["clean_t"]

2        biographical sketches thf leadin men present a...
7        jiy iid yic practical exercises learning thjii...
10       pray ner ganpksg ingg almight sed itn churchec...
14       chd urra gih rbl icd ril iii nlill ulv bas slr...
17       lingux latin compendiarius compendious latin t...
                               ...                        
15890    brol catechism health selected translated germ...
15893    thre acts performed eatr royal drry lane connu...
15898    volume author works containing conduct allies ...
15904    advice people general regard health particular...
15908    maid bath performed theatr royal hay market sa...
Name: clean_t, Length: 4052, dtype: object

In [43]:
stop_words_l = stopwords.words("english")
#list(stop_words_l)

In [44]:
import os

In [45]:
os.name

'posix'

In [46]:
import platform

In [47]:
platform.system()

'Linux'

In [48]:
platform.release()

'4.18.0-372.46.1.el8.x86_64'

In [49]:
!{sys.executable} -m pip show contextualized-topic-models

Name: contextualized-topic-models
Version: 2.5.0
Summary: Contextualized Topic Models
Home-page: https://github.com/MilaNLProc/contextualized-topic-models
Author: Federico Bianchi
Author-email: f.bianchi@unibocconi.it
License: MIT license
Location: /users/sebarodr/.local/lib/python3.10/site-packages
Requires: gensim, ipython, ipywidgets, matplotlib, numpy, scipy, sentence-transformers, torch, torchvision, tqdm, wordcloud
Required-by: 


In [50]:
!{sys.executable} -m pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: dask-glm, dask-ml, esda, hdbscan, imbalanced-learn, mapclassify, open3d, pyLDAvis, pynndescent, segregation, sentence-transformers, spopt, spreg, umap-learn


In [53]:
def create_chunks(row):
    arr = row['clean_t'].split()
    n = 128
    chunks = [" ".join(arr[i:i+n]) for i in range(0, len(arr), n)]
    return pd.DataFrame({'chunk': chunks ,'document_id':row['document_id']})

In [57]:
english_docs.to_csv("cleaned_text_dataframe.csv",index=False)

In [9]:
english_docs = pd.read_csv("cleaned_text_dataframe.csv")

In [58]:
a = english_docs.apply(lambda x: create_chunks(x),axis=1)

In [59]:
chunks_df = pd.DataFrame()
for i,row in a.items():
    chunks_df = pd.concat([chunks_df,row])

In [60]:
chunks_df

,chunk,document_id
0,biographical sketches thf leadin men present a...,1243400400
1,many political luminaries many individuals vir...,1243400400
2,future occasion manifested first day elecion d...,1243400400
3,day still fresh memory every one heart would c...,1243400400
4,concerted plan brifibt chabot merlin permit te...,1243400400
...,...,...
53,disappointed lady weel maifter flent wul ling ...,0814900800
54,procure able defenders good wit nesses wanted ...,0814900800
55,thou sand good guineas bargain rack sir sir ex...,0814900800
56,cat cloaths coachman billy button afford new s...,0814900800


In [61]:
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords as stop_words
from gensim.utils import deaccent

In [62]:
documents = chunks_df["chunk"].tolist()
stopwords_set = set(list(stop_words_l))
vocabulary_size = 2000
max_df = 1.0
min_words = 1
remove_numbers = True

In [63]:
def preprocess():
    """
    Note that if after filtering some documents do not contain words we remove them. That is why we return also the
    list of unpreprocessed documents.
    :return: preprocessed documents, unpreprocessed documents and the vocabulary list
    """
    preprocessed_docs_tmp = documents
    preprocessed_docs_tmp = [deaccent(doc.lower()) for doc in preprocessed_docs_tmp]
    preprocessed_docs_tmp = [doc.translate(
        str.maketrans(string.punctuation, ' ' * len(string.punctuation))) for doc in preprocessed_docs_tmp]
    if remove_numbers:
        preprocessed_docs_tmp = [doc.translate(str.maketrans("0123456789", ' ' * len("0123456789")))
                                 for doc in preprocessed_docs_tmp]
    preprocessed_docs_tmp = [' '.join([w for w in doc.split() if len(w) > 0 and w not in stopwords_set])
                             for doc in preprocessed_docs_tmp]

    vectorizer = CountVectorizer(max_features=vocabulary_size, max_df=max_df)
    vectorizer.fit_transform(preprocessed_docs_tmp)
    temp_vocabulary = set(vectorizer.get_feature_names_out())

    preprocessed_docs_tmp = [' '.join([w for w in doc.split() if w in temp_vocabulary])
                             for doc in preprocessed_docs_tmp]

    preprocessed_docs, unpreprocessed_docs, retained_indices = [], [], []
    for i, doc in enumerate(preprocessed_docs_tmp):
        if len(doc) > 0 and len(doc) >= min_words:
            preprocessed_docs.append(doc)
            unpreprocessed_docs.append(documents[i])
            retained_indices.append(i)

    vocabulary = list(set([item for doc in preprocessed_docs for item in doc.split()]))

    return preprocessed_docs, unpreprocessed_docs, vocabulary, retained_indices

In [64]:
#sp = WhiteSpacePreprocessingStopwords(english_docs["clean_t"].tolist(), stopwords_list=list(stop_words_l))
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = preprocess()

In [65]:
#stop_words = stopwords.words("english")
#english_docs["tokens"] =english_docs["text"].map(lambda x: clean_text(x, word_tokenize, stop_words))

In [66]:
preprocessed_documents[:1]

['men present nature quantity government proper mar attend first years life pas fed honourable writers humble genius lately fellow ing called ther people remedy wants well art favour clergy influence returned fixed deep bosom fury four years ment men ring even bloody generous qualities heart contain']

In [67]:
vocab[:10]

['treated',
 'ear',
 'followed',
 'road',
 'born',
 'told',
 'family',
 'fancy',
 'oath',
 'religion']

In [68]:
unpreprocessed_corpus

['biographical sketches thf leadin men present ada rffairs frantce understand nature quantity government proper mar ncceffary attend charater parne rigcts edinburgh dickson creech fairbairn mjdcc xciii idl trl citizen hion first years mons pethion life pas fed pracice honourable functions lowest class attornies writers moving humble sphere talents circuin fcribed individuals reaped benefits genius lately extended fellow citizens louis xvi liften ing complaints fubjees called toge ther representatives people concert remedy wants hypo crite well managed art insinuated favour clergy influence returned firt national assembly member chartres adder fixed deep fting bosom softered relenting relenting animosity fanned exterminating fury four years persecution gainft class citizens flanding monu ment gratitude massacre thousands unfortunate men ring mayorality connived even mented inscribed bloody characters generous qualities heart firt national assembly contain',
 'many political luminaries m

In [69]:
len(unpreprocessed_corpus)

1651190

In [70]:
len(preprocessed_documents)

1651190

In [71]:
tp = TopicModelDataPreparation("paraphrase-multilingual-mpnet-base-v2")

training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus[:59831], text_for_bow=preprocessed_documents[:59831])

Batches:   0%|          | 0/300 [00:00<?, ?it/s]

In [72]:
tp.vocab[:10]

array(['able', 'abroad', 'absolute', 'accept', 'according', 'accordingly',
       'account', 'accounts', 'acquaintance', 'acquainted'], dtype=object)

In [73]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=768, n_components=15, num_epochs=20)
ctm.fit(training_dataset) # run the model

/users/sebarodr/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Epoch: [20/20]	 Seen Samples: [1195520/1196620]	Train Loss: 395.896014260531	Time: 0:00:14.251512: : 20it [05:08, 15.42s/it]  
100%|██████████| 935/935 [00:12<00:00, 76.62it/s] 


In [74]:
doc_topic_dist = ctm.get_doc_topic_distribution(training_dataset)

100%|██████████| 935/935 [00:11<00:00, 83.96it/s] 


In [75]:
ctm.get_most_likely_topic(doc_topic_dist)

array([25147,  2693, 11722, 23816, 39901, 59782, 50112, 18793, 28523,
       18634, 27672,  9370, 40958, 21229, 51777])

In [76]:
ctm.get_topics()

defaultdict(list,
            {0: ['would',
              'mrs',
              'lady',
              'sir',
              'never',
              'could',
              'child',
              'saw',
              'knew',
              'told'],
             1: ['instance',
              'speaks',
              'blessed',
              'confess',
              'surely',
              'angel',
              'faults',
              'chosen',
              'satisfied',
              'bishop'],
             2: ['thy',
              'god',
              'thou',
              'thee',
              'lord',
              'holy',
              'jesus',
              'christ',
              'unto',
              'love'],
             3: ['thou',
              'thy',
              'thee',
              'death',
              'kill',
              'heart',
              'rich',
              'foul',
              'like',
              'king'],
             4: ['mind',
              'nature',
        

In [77]:
ctm.get_topic_lists(5)

[['would', 'mrs', 'lady', 'sir', 'never'],
 ['instance', 'speaks', 'blessed', 'confess', 'surely'],
 ['thy', 'god', 'thou', 'thee', 'lord'],
 ['thou', 'thy', 'thee', 'death', 'kill'],
 ['mind', 'nature', 'thing', 'motion', 'ideas'],
 ['may', 'patient', 'ought', 'disease', 'proper'],
 ['court', 'plaintiff', 'cafe', 'tenant', 'law'],
 ['instance', 'faults', 'speaks', 'owe', 'surely'],
 ['said', 'shall', 'fame', 'part', 'time'],
 ['god', 'men', 'man', 'vol', 'mankind'],
 ['take', 'two', 'little', 'salt', 'put'],
 ['city', 'town', 'plin', 'river', 'near'],
 ['thro', 'ring', 'deep', 'around', 'round'],
 ['king', 'lord', 'henry', 'good', 'edward'],
 ['army', 'parliament', 'great', 'king', 'nation']]

In [78]:
lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset,200)

100%|██████████| 935/935 [00:16<00:00, 58.42it/s] 


In [79]:
import pyLDAvis as vis
movies_pd = vis.prepare(**lda_vis_data)
vis.display(movies_pd)

/users/sebarodr/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [80]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) 

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/users/sebarodr/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 935/935 [00:10<00:00, 87.31it/s] 


In [81]:
preprocessed_documents[0]

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'men present nature quantity government proper mar attend first years life pas fed honourable writers humble genius lately fellow ing called ther people remedy wants well art favour clergy influence returned fixed deep bosom fury four years ment men ring even bloody generous qualities heart contain'

In [82]:
import numpy as np
topic_number = np.argmax(topics_predictions[0])

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [83]:
ctm.get_topic_lists(5)[topic_number]

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['god', 'men', 'man', 'vol', 'mankind']

In [89]:
preprocessed_documents[0],chunks_df.iloc[0]['chunk']

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


('men present nature quantity government proper mar attend first years life pas fed honourable writers humble genius lately fellow ing called ther people remedy wants well art favour clergy influence returned fixed deep bosom fury four years ment men ring even bloody generous qualities heart contain',
 'biographical sketches thf leadin men present ada rffairs frantce understand nature quantity government proper mar ncceffary attend charater parne rigcts edinburgh dickson creech fairbairn mjdcc xciii idl trl citizen hion first years mons pethion life pas fed pracice honourable functions lowest class attornies writers moving humble sphere talents circuin fcribed individuals reaped benefits genius lately extended fellow citizens louis xvi liften ing complaints fubjees called toge ther representatives people concert remedy wants hypo crite well managed art insinuated favour clergy influence returned firt national assembly member chartres adder fixed deep fting bosom softered relenting rele

In [91]:
chunks_df.iloc[0]['document_id']

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'1243400400'

In [92]:
english_docs[english_docs['document_id'] == '1243400400']

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,document_id,work_id,main_category,sub_category,text,ecco_id_x,estc_id,ecco_part,ecco_module,ecco_full_title,...,ecco_pages,ecco_languge,ecco_nr_paragraphs,ecco_id_y,document_part_type,document_part_number,document_part_id,document_part_nr_characters,document_part_nr_tokens,clean_t
2,1243400400,X-biographical sketches of some of the leading...,4,10,biographical sketches of some of thf leadin me...,1243400400,N31961,ecco1,Social Sciences,Biographical sketches of some of the leading m...,...,62.0,English,226.0,1243400400,body,3.0,214495.0,73568.0,12607.0,biographical sketches thf leadin men present a...


In [94]:
english_docs['document_id'].unique()

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['1243400400', '1588400300', '1628902300', ..., '0098000205',
       '0884900100', '0814900800'], dtype=object)

In [101]:
chunks_df.reset_index(drop = True,inplace=True)

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [106]:
unique_ids = chunks_df.iloc[:59831]['document_id'].unique()

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [121]:
top = []
for i in unique_ids:
    idx = chunks_df[chunks_df['document_id'] == i].index
    idx = idx[idx < 59831]
    top.append(np.argmax(topics_predictions[idx],axis=1))

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [122]:
topicos = []
for i in top:
    topi, count = np.unique(topics_pred_id, return_counts=True)
    topicos.append(topi[np.argmax(count)])

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [128]:
english_docs[english_docs['document_id'].isin(unique_ids)]['main_category'].value_counts()

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2    36
6    17
4    16
1    16
0    15
9    10
5     9
8     6
3     3
7     1
Name: main_category, dtype: int64

In [123]:
topicos

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5]

In [117]:
chunks_df[chunks_df['document_id'] == unique_ids[-1]].index.shape

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(3099,)

In [119]:
idx.shape

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2319,)

In [115]:
topics_pred_id.shape

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2319,)

In [120]:
np.unique(topics_pred_id, return_counts=True)

/users/sebarodr/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(array([ 0,  1,  2,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14]),
 array([  14,    4,    1,   58, 2039,    2,    3,   99,   90,    3,    2,
           1,    3]))